# Temporal Evolution Notebook Tutorial 

This is a tutorial of Section 5, aiming to help the user to understand how data is loaded and ploted. To make the tutorial easier, the code found here is a simplification of the original script (Tropidash_backbone.ipynb). Here, only the widgets of Section 5 are found and the initial date of the forecast and the cyclone are set to 2023-09-07 and LEE respectively. Please notice that only two days of the cyclone are considered in this tutorial but in the original script all the forecasted days of the cyclone are considered. 

In the original script, cyclone and atmospheric data are downloaded in Sections 1 and 2 respectivelly. Here we add the code fragments from Sections 1 and 2 that retrieve the data needed in Section 5. From Section 1: average of the ensembles forecasted (track locations_avg). From Section 2: accumulated precipitation, skin temperature, mean sea level pressure and probability of having wind gusts > 25m/s at 10m. 

# Libraries import

In [1]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import display

from ecmwf.opendata import Client
import numpy as np
import ipyleaflet
from ipyleaflet import Map, Marker

import ipywidgets
import xarray as xr
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt

from bqplot import Lines, Figure, LinearScale, Axis, DateScale
import bqplot
from datetime import datetime

# Import utilities functions from TropiDash main folder
import os
cwd = os.getcwd()
cwd = os.path.sep.join(cwd.split(os.path.sep)[:-1])
import sys
sys.path.insert(1, os.path.join(cwd, 'TropiDash'))
import utils_tracks as tracks

from utils_TemporalEvolution import *
from utils_atm import *

# Initial parameters obtained from the initialization widget

In [ ]:
# Selection of the cyclone
cyclone_ID = '13L'

# Selection of the starting date of the forecast
start_date = datetime(2023, 9, 7, 0, 0)

# Retrieve cyclone data (Section 1)

In [2]:
# Load the ECMWF forecast data
df_storms_forecast = tracks.create_storms_df(start_date)

# Select the data of storm LEE (Storm Identifier: 13L)
df_storm_forecast = df_storms_forecast[df_storms_forecast.stormIdentifier == cyclone_ID]

locations_avg, timesteps_avg, pressures_avg, wind_speeds_avg = tracks.mean_forecast_track(df_storm_forecast)

# Retrieve atmospheric data (Section 2)

Here, the steps of the forecast to be downloaded are defined.

In [3]:
cyclone_days = 2 # in the original script this the number of the forecasted days of the cyclone 

#steps for the variable probability of having wind gusts > 25m/s at 10m
windprobsteps = [f"{12 * i}-{ 12 * i + 24}" for i in range(2*cyclone_days-1)]
windprobsteps.insert(0, "0-24")

# dictionary of the steps for the wind gust prob. with the key "10fgg25" and of the 
# steps for the other variables with the key "base"
stepsdict = {
               "base": list(np.arange(12, 240, 12)[0:2*cyclone_days]),
               "10fgg25": windprobsteps
            }
#list of variables to visualize in Section 5
variables = ['msl', 'skt', 'tp', '10fgg25'] 

Download data

In [ ]:
fnames = dwnl_atmdata_step(variables, stepsdict, start_date);

Load data

In [ ]:
vardict = load_atmdata(variables, fnames)

The following code is specific of Section 5

# Variable initialization

In [4]:
#first cyclone position
initial_latlon = locations_avg[0] #(38, -90) 

# Steps to download 
steps_to_download = stepsdict['base']
steps_to_download2 = stepsdict['10fgg25']

# Initial date
initial_date = start_date.strftime("%Y%m%d")

# Average cyclone track
avg_track = locations_avg

# Load atmospheric data in the formating needed for Section 5

In [5]:
### LOAD DATA
data_allpoints = get_allvars_allpoints(initial_date, steps_to_download, steps_to_download2)

# Change units of skin temperature  K to ºC
data_allpoints['skt']['Value'] = data_allpoints['skt']['Value'] - 273.15

# Change units of mean sea level pressure from Pa to hPa
data_allpoints['msl']['Value'] = data_allpoints['msl']['Value']/100    

# Plotting

### Create background map

In [6]:
m = Map(
    center=[initial_latlon[0], initial_latlon[1]+50],
    basemap=ipyleaflet.basemaps.Esri.WorldTopoMap,
    zoom = 2,
)

### Create plots of the initial marker location

In [7]:
p1, p2, p3, p4 = get_initial_plot(data_allpoints, initial_date, initial_latlon, steps_to_download)    

### Transform the plots into a widget from the plot

In [10]:
# Create widget layout
item_layout = ipywidgets.Layout(overflow_y='scroll', width='350px', height='350px',
                               flex_flow='column', display='block')

# Insert plots into the widget
main_figure = widgets.Box(children=[p1, p2, p3, p4], layout=item_layout,  width='350px', height='350px')

# Place the widget at the bottom-right of the background map
widget_control1 = ipyleaflet.WidgetControl(widget=main_figure, position="bottomright")
m.add(widget_control1)

Map(center=[15.150012821314707, 2.9538448747324253], controls=(ZoomControl(options=['position', 'zoom_in_text'…

### Add a marker to the background map

In [11]:
# Create marker
marker = Marker(location=initial_latlon, draggable=True, name = 'Position') 

# Add marker to the background map
m.add_layer(marker)

# Define the actions occurring when the marked is moved into a new position
marker.on_move(handle_move(data_allpoints, steps_to_download))

### Add the average track to the bacground map

In [13]:
# Create the AntPath
avg_track_antpath = ipyleaflet.AntPath(locations = avg_track, color = "red")

# Add the AntPath to the background map
m.add_layer(avg_track_antpath)

### Display the plot of Section 5

In [14]:
display(m)

Map(bottom=668.0, center=[15.150012821314707, 2.9538448747324253], controls=(ZoomControl(options=['position', …